# Exercises

Within your codeup-data-science directory, create a new repo named spark-exercises. This will be where you do your work for this module. Create a repository on GitHub with the same name, and link your local repository to GitHub.

- Save this work in your spark-exercises repo. Then add, commit, and push your changes.

- Create a jupyter notebook or python script named spark101 for this exercise.


In [1]:
import pyspark
# once per notebook
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [2]:
import multiprocessing
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.functions import col, expr
from pyspark.sql.functions import sqrt
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean
from pyspark.sql.functions import when

1) Create a spark data frame that contains your favorite programming languages.
   - The name of the column should be language
   - View the schema of the dataframe
   - Output the shape of the dataframe
   - Show the first 5 records in the dataframe

In [17]:
spark.createDataFrame(["Python","Javascript","C++", "R", "Go", "Kotlin", "Swift", "PHP"], "string").toDF("language").printSchema()

root
 |-- language: string (nullable = true)



In [15]:
spark.createDataFrame(["Python","Javascript","C++", "R", "Go", "Kotlin", "Swift", "PHP"], "string").toDF("language").shape()

(8, 1)

In [4]:
favorite_programming_languages = spark.createDataFrame(["Python","Javascript","C++", "R", "Go", "Kotlin", "Swift", "PHP"], "string").toDF("language").show(5)

+----------+
|  language|
+----------+
|    Python|
|Javascript|
|       C++|
|         R|
|        Go|
+----------+
only showing top 5 rows



2) Load the mpg dataset as a spark dataframe.

   - Create 1 column of output that contains a message like the one below:

        - The 1999 audi a4 has a 4 cylinder engine.

        - For each vehicle.

   - Transform the trans column so that it only contains either manual or auto.

In [19]:
from pydataset import data

In [21]:
mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [ ]:
The {year} {manufacturer} {model} has a {cyl} cylinder engine

In [32]:
from pyspark.sql.functions import lit

In [44]:
mpg.select((concat(lit("The "),mpg.year,lit(" "), mpg.manufacturer,lit(" "), mpg.model, lit(" has a "), mpg.cyl, lit(" cylinder engine")).alias("description"))).show(5)

+--------------------+
|         description|
+--------------------+
|The 1999 audi a4 ...|
|The 1999 audi a4 ...|
|The 2008 audi a4 ...|
|The 2008 audi a4 ...|
|The 1999 audi a4 ...|
+--------------------+
only showing top 5 rows



In [115]:
mpg.select(mpg.trans, when(mpg.trans.startswith("a"), "auto").otherwise("manual").alias("transmission")).show(4)

+----------+------------+
|     trans|transmission|
+----------+------------+
|  auto(l5)|        auto|
|manual(m5)|      manual|
|manual(m6)|      manual|
|  auto(av)|        auto|
+----------+------------+
only showing top 4 rows



3) Load the tips dataset as a spark dataframe.

   - What percentage of observations are smokers?
   - Create a column that contains the tip percentage
   - Calculate the average tip percentage for each combination of sex and smoker.


In [118]:
tips = spark.createDataFrame(data("tips"))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



4) Use the seattle weather dataset referenced in the lesson to answer the questions below.

   - Convert the temperatures to farenheight.
   - Which month has the most rain, on average?
   - Which year was the windiest?
   - What is the most frequent type of weather in January?
   - What is the average high and low temperature on sunny days in July in 2013 and 2014?
   - What percentage of days were rainy in q3 of 2015?
   - For each year, find what percentage of days it rained (had non-zero precipitation).

In [277]:
from vega_datasets import data
seattle = spark.createDataFrame(data.seattle_weather())
seattle.show(7)

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06 00:00:00|          2.5|     4.4|     2.2| 2.2|   rain|
|2012-01-07 00:00:00|          0.0|     7.2|     2.8| 2.3|   rain|
+-------------------+-------------+--------+--------+----+-------+
only showing top 7 rows



In [278]:
temp_max_f = (col("temp_max") * 9/5) + 32

In [279]:
temp_min_f = (col("temp_min") * 9/5) + 32

In [280]:
seattle.select(
    seattle.date,
    seattle.precipitation,
    seattle.temp_max.alias("temp_max_c"),
    seattle.temp_min.alias("temp_min_c"),
    temp_max_f.alias("temp_max_f"),
    temp_min_f.alias("temp_min_f"),
    seattle.wind,
    seattle.weather
).show(7)

+-------------------+-------------+----------+----------+----------+----------+----+-------+
|               date|precipitation|temp_max_c|temp_min_c|temp_max_f|temp_min_f|wind|weather|
+-------------------+-------------+----------+----------+----------+----------+----+-------+
|2012-01-01 00:00:00|          0.0|      12.8|       5.0|     55.04|      41.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|      10.6|       2.8|     51.08|     37.04| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|      11.7|       7.2|     53.06|     44.96| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|      12.2|       5.6|     53.96|     42.08| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|       8.9|       2.8|     48.02|     37.04| 6.1|   rain|
|2012-01-06 00:00:00|          2.5|       4.4|       2.2|     39.92|     35.96| 2.2|   rain|
|2012-01-07 00:00:00|          0.0|       7.2|       2.8|     44.96|     37.04| 2.3|   rain|
+-------------------+-------------+----------+----------+----------+--

In [281]:
from pyspark.sql.functions import to_timestamp

In [282]:
from pyspark.sql.functions import month, year, quarter

In [283]:
df = seattle.select(to_timestamp(seattle.date, 'yyyy-MM-dd HH:mm:ss').alias("dt"), seattle.wind, seattle.weather).show(5)

+-------------------+----+-------+
|                 dt|wind|weather|
+-------------------+----+-------+
|2012-01-01 00:00:00| 4.7|drizzle|
|2012-01-02 00:00:00| 4.5|   rain|
|2012-01-03 00:00:00| 2.3|   rain|
|2012-01-04 00:00:00| 4.7|   rain|
|2012-01-05 00:00:00| 6.1|   rain|
+-------------------+----+-------+
only showing top 5 rows



In [284]:
from pyspark.sql.functions import dayofmonth

In [327]:
seattle.select(
    seattle.date,
    year("date").alias('year'), 
    month("date").alias('month'), 
    dayofmonth("date").alias('day'),
    seattle.precipitation,
    seattle.temp_max.alias("temp_max_c"),
    seattle.temp_min.alias("temp_min_c"),
    temp_max_f.alias("temp_max_f"),
    temp_min_f.alias("temp_min_f"),
    seattle.wind,
    seattle.weather
).show(7)

+-------------------+----+-----+---+-------------+----------+----------+----------+----------+----+-------+
|               date|year|month|day|precipitation|temp_max_c|temp_min_c|temp_max_f|temp_min_f|wind|weather|
+-------------------+----+-----+---+-------------+----------+----------+----------+----------+----+-------+
|2012-01-01 00:00:00|2012|    1|  1|          0.0|      12.8|       5.0|     55.04|      41.0| 4.7|drizzle|
|2012-01-02 00:00:00|2012|    1|  2|         10.9|      10.6|       2.8|     51.08|     37.04| 4.5|   rain|
|2012-01-03 00:00:00|2012|    1|  3|          0.8|      11.7|       7.2|     53.06|     44.96| 2.3|   rain|
|2012-01-04 00:00:00|2012|    1|  4|         20.3|      12.2|       5.6|     53.96|     42.08| 4.7|   rain|
|2012-01-05 00:00:00|2012|    1|  5|          1.3|       8.9|       2.8|     48.02|     37.04| 6.1|   rain|
|2012-01-06 00:00:00|2012|    1|  6|          2.5|       4.4|       2.2|     39.92|     35.96| 2.2|   rain|
|2012-01-07 00:00:00|2012|  

In [353]:
seattle.select(
    month("date").alias('month'), 
    seattle.precipitation,
    seattle.wind,
    seattle.weather,
).show(7)

+-----+-------------+----+-------+
|month|precipitation|wind|weather|
+-----+-------------+----+-------+
|    1|          0.0| 4.7|drizzle|
|    1|         10.9| 4.5|   rain|
|    1|          0.8| 2.3|   rain|
|    1|         20.3| 4.7|   rain|
|    1|          1.3| 6.1|   rain|
|    1|          2.5| 2.2|   rain|
|    1|          0.0| 2.3|   rain|
+-----+-------------+----+-------+
only showing top 7 rows



In [354]:
seattle.show(5)

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|     8.9|     2.8| 6.1|   rain|
+-------------------+-------------+--------+--------+----+-------+
only showing top 5 rows



In [363]:
seattle.filter(month("date") == 1)
    .withColumn("avg", (seattle.precipitation)/len(seattle.precipitation)).show()


IndentationError: unexpected indent (<ipython-input-363-dbaa2f3d79da>, line 2)

In [ ]:
(
    weather.filter(year("date") == 2013)
    .withColumn("freezing_temps", (weather.temp_avg <= 0).cast("int"))
    .withColumn("month", month("date"))
    .groupBy("month")
    .agg(sum("freezing_temps").alias("no_of_days_with_freezing_temps"))
    .sort("month")
    .show()
)